In [ ]:


import os
import pandas as pd
import numpy as np
import glob
import itertools 
import subprocess

from bayes_opt import BayesianOptimization, UtilityFunction
from bayes_opt.observer import JSONLogger
from bayes_opt.event import Events, DEFAULT_EVENTS
from bayes_opt.util import load_logs

pbounds = {'car_intercept': (-5, 5),\
         'walk_transit_intercept': (-5, 5),\
         'drive_transit_intercept': (-5, 5),\
         'ride_hail_transit_intercept': (-5, 5),\
         'ride_hail_intercept': (-5, 5),\
         'ride_hail_pooled_intercept': (-5, 5), \
         'walk_intercept': (-5, 5),\
         'bike_intercept': (-5, 5)}


new_dict = {}

starter = ['car_intercept','walk_transit_intercept','drive_transit_intercept',\
         'ride_hail_transit_intercept','ride_hail_intercept','ride_hail_pooled_intercept',\
         'walk_intercept','bike_intercept']

Repo='/home/berkeleylab/Repository/beam/'
Current='/home/berkeleylab/Calib_documentation/TUNE_integrate/TUNE_test'



In [ ]:


def ext_change(param):    
    if param == 'edit':
        for filename in glob.iglob(os.path.join(Repo, 'test/input/beamville/beam.conf')):
            os.rename(filename, filename[:-9] + 'beam.txt')
    elif param == 'save':
        for filename in glob.iglob(os.path.join(Repo, 'test/input/beamville/beam.txt')):
            os.rename(filename, filename[:-8] + 'beam.conf')

def change_conf(new_dict):
    
    hyperparams = [new_dict[k] for k in starter]
    
    with open(Repo+'test/input/beamville/beam.txt', 'r') as fin:
        file_text=fin.readlines()
    
    file_text[24] = file_text[24].split('=',1)[0]+'= '+hyperparams[0]+' \n'
    file_text[25] = file_text[25].split('=',1)[0]+'= '+hyperparams[1]+' \n'
    file_text[26] = file_text[26].split('=',1)[0]+'= '+hyperparams[2]+' \n'
    file_text[27] = file_text[27].split('=',1)[0]+'= '+hyperparams[3]+' \n'
    file_text[28] = file_text[28].split('=',1)[0]+'= '+hyperparams[4]+' \n'
    file_text[29] = file_text[29].split('=',1)[0]+'= '+hyperparams[5]+' \n'
    file_text[30] = file_text[30].split('=',1)[0]+'= '+hyperparams[6]+' \n'
    file_text[31] = file_text[31].split('=',1)[0]+'= '+hyperparams[7]+' \n'
    
    with open(Repo+'test/input/beamville/beam.txt', 'w') as fini:
        for i in file_text:
            fini.write(i)
        
      
def run_beam():
    os.chdir(Repo)
    subprocess.call([Repo+'runme.sh'])
    os.chdir(Current)

def all_subdirs_of(b='/home/berkeleylab/Repository/beam/output/beamville/'):
  result = []
  for d in os.listdir(b):
    bd = os.path.join(b, d)
    if os.path.isdir(bd): result.append(bd)
  return result

def beamville(new_dict):
    ext_change('edit')
    change_conf(new_dict)
    ext_change('save') 
    run_beam()
    latest_subdir = max(all_subdirs_of(), key=os.path.getmtime)
    df = pd.read_csv(latest_subdir+"/referenceRealizedModeChoice.csv").iloc[[0, -1]].drop(['iterations'], axis=1)
    acc = (df.iloc[0] - df.iloc[-1]).abs().sum()/df.iloc[0].sum()
    return acc


In [ ]:

optimizer = BayesianOptimization(f=None, pbounds=pbounds,\
                                verbose=2, random_state=1)
utility = UtilityFunction(kind='ucb', kappa=2.5, xi= 0.01)
next_point_to_probe = optimizer.suggest(utility)
target = beamville(**next_point_to_probe)
optimizer.register(params=next_point_to_probe, target=target)

for _ in range(5):
    next_point = optimizer.suggest(utility)
    target = black_box_function(**next_point)
    optimizer.register(params=next_point, target= target)
    print(target, next_point)
    
print(optimizer.max)
